<a href="https://colab.research.google.com/github/meti-94/OpenQA/blob/main/relation_classification_with_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install necessary libraries 

In [ ]:
# Install main transformers library
!pip install transformers -q
!rm -rf OpenQA/
!git clone https://github.com/meti-94/OpenQA.git

Cloning into 'OpenQA'...
remote: Enumerating objects: 129, done.
remote: Counting objects: 100% (129/129), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 129 (delta 37), reused 105 (delta 21), pack-reused 0
Receiving objects: 100% (129/129), 88.92 MiB | 26.45 MiB/s, done.
Resolving deltas: 100% (37/37), done.


# Import libraries

In [ ]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from transformers import AutoConfig, AutoTokenizer, AutoModel, BertForSequenceClassification
from sklearn.metrics import classification_report
from sklearn import preprocessing

# load up data to classifiy

In [ ]:
## loading data
## X --> Texts
## y --> Labels
le = preprocessing.LabelEncoder()
df_train = pd.read_excel('/content/OpenQA/data/freebase/train_useful_records.xlsx')
df_train['label'] = le.fit_transform(df_train.relation_type.to_list())
df_valid = pd.read_excel('/content/OpenQA/data/freebase/valid_useful_records.xlsx')
df_valid['label'] = le.transform(df_valid.relation_type.to_list())
df_test = pd.read_excel('/content/OpenQA/data/freebase/test_useful_records.xlsx')
df_test['label'] = le.transform(df_test.relation_type.to_list())


ValueError: ignored

In [ ]:
df_train.head()

,Unnamed: 0,Unnamed: 0.1,token_matrix,Question,tokenized_question,Answer,first_entity_ids,second_entity_ids,relation_type,entity,answer_mid,relation_span
0,0,0,"[101, 2054, 2003, 1996, 2338, 1041, 2055, 102]",what is the book e about,"['[CLS]', 'e', '[SEP]']",fb:m.04whkz5,"[101, 1041, 102]","[101, 102]",fb:book.written_work.subjects,e,fb:m.04whkz5,"[0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,1,1,"[101, 2000, 2054, 2713, 2515, 1996, 2713, 2650...",to what release does the release track cardiac...,"['[CLS]', 'cardiac', 'arrest', '[SEP]']",fb:m.0tp2p24,"[101, 15050, 6545, 102]","[101, 102]",fb:music.release_track.release,cardiac arrest,fb:m.0tp2p24,"[0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, ..."
2,2,2,"[101, 2054, 2406, 2001, 1996, 2143, 1996, 7016...",what country was the film the debt from,"['[CLS]', 'the', 'debt', '[SEP]']",fb:m.04j0t75,"[101, 1996, 7016, 102]","[101, 102]",fb:film.film.country,the debt,fb:m.04j0t75,"[0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, ..."
3,3,3,"[101, 2054, 2774, 2031, 2053, 8569, 2080, 1057...",what songs have nobuo uematsu produced ?,"['[CLS]', 'no', '##bu', '##o', 'u', '##ema', '...",fb:m.0ftqr,"[101, 2053, 8569, 2080, 1057, 14545, 10422, 102]","[101, 102]",fb:music.producer.tracks_produced,nobuo uematsu,fb:m.0ftqr,"[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, ..."
4,4,4,"[101, 2040, 2550, 6574, 1011, 19330, 13700, 10...",who produced eve-olution ?,"['[CLS]', 'eve', '-', 'ol', '##ution', '[SEP]']",fb:m.036p007,"[101, 6574, 1011, 19330, 13700, 102]","[101, 102]",fb:music.release.producers,eve-olution,fb:m.036p007,"[0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."


# Loading BERT model and tokenizer model

In [ ]:

# v1.0
config = AutoConfig.from_pretrained("prajjwal1/bert-tiny")
tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-tiny")
model = BertForSequenceClassification.from_pretrained("prajjwal1/bert-tiny", num_labels=df_train.relation_type.nunique())


Downloading:   0%|          | 0.00/285 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/16.9M [00:00<?, ?B/s]

Some weights of the model checkpoint at prajjwal1/bert-tiny were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initia

# Creating dataset and dataloader for X-y

In [ ]:
train_encodings = tokenizer(df_train.Question.to_list(), truncation=True, padding=True, max_length=200)
val_encodings = tokenizer(df_valid.Question.to_list(), truncation=True, padding=True, max_length=200)

In [ ]:
# convert raw text file to proper dataset object (based on task)
import torch

class ClassificationDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        # initialization
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        # slicing method X[index]
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = ClassificationDataset(train_encodings, df_train.relation_type.to)
valid_dataset = ClassificationDataset(val_encodings, valid_labels)

NameError: ignored

In [ ]:
# transformers API for train :)
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=5,              # total number of training epochs
    per_device_train_batch_size=36,  # batch size per device during training
    per_device_eval_batch_size=36,   # batch size for evaluation
    # warmup_steps=500,                # number of warmup steps for learning rate scheduler
    # weight_decay=0.01,               # strength of weight decay
    learning_rate= 5e-6,
    adam_epsilon = 1e-8, 
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    do_eval=True,
    evaluation_strategy = 'epoch'
    
    
)
# training_args.evaluation_strategy = EvaluationStrategy.EPOCH
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=valid_dataset,          # evaluation dataset

)

trainer.train()

Epoch,Training Loss,Validation Loss
1,0.400364,0.385962


KeyboardInterrupt: ignored

In [ ]:
# reading Test Data
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=50)
test_dataset = IMDbDataset(test_encodings, test_labels)

# Predicting on test set

In [ ]:
pred = trainer.predict(test_dataset)

# Evaluation

In [ ]:
print(classification_report(pred.label_ids, pred.predictions.argmax(-1)))

              precision    recall  f1-score   support

           0       0.94      0.82      0.88      1196
           1       0.55      0.81      0.65       320

    accuracy                           0.82      1516
   macro avg       0.74      0.81      0.76      1516
weighted avg       0.86      0.82      0.83      1516



In [ ]:
trainer.save_model("./classifier")

In [ ]:
cp -r classifier/ drive/MyDrive/